In [49]:
import pandas as pd 
import librosa 
import IPython.display 
import os 
from tqdm import tqdm 

from data_preparation.alignment import force_align


In [50]:
root_path='/root/suyash/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset'
train_csv='/root/suyash/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset/sample_bhashini_train.csv'
dev_csv='/root/suyash/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset/sample_bhashini_dev.csv'

In [51]:
df_1=pd.read_csv(os.path.join(root_path,train_csv))
df_2=pd.read_csv(os.path.join(root_path,dev_csv))
df_1=df_1.drop(columns=['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'level_0', 'index'])
df_2=df_2.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'level_0',
       'index','sampled_words'])
df=pd.concat([df_1,df_2])
df=df.reset_index()
correct_df=df[df['transcript']==df['bhashini_transcript']]
duplet_idx_list=[]
for i in range(len(df)):
    if len(df['bhashini_transcript'][i].split(' '))==2:
        duplet_idx_list.append(i)

filtered_df=df.iloc[duplet_idx_list]
filtered_df=filtered_df.reset_index()

In [76]:
len(filtered_df)

21869

In [77]:
list=[]
for i in range(len(filtered_df)):
    if len(filtered_df['bhashini_transcript'][i].split(' '))<2:
        list.append(i)

In [78]:
len(list)

0

In [79]:
import torch 
import numpy as np
import json  
import os 
from scipy.io.wavfile import write
import pandas as pd 

from data_preparation.model import Model 
from data_preparation.audio import load_audio,SAMPLE_RATE
from data_preparation.utils import Point,Segment
from data_preparation.alignment import compose_graph, backtrack, _load_model,force_align

In [82]:
from tqdm import tqdm 
model=_load_model()

def compute_t_j(df):

    t_list=[]
    j_list=[]

    for i in tqdm(range(len(df))):
        audio_path=os.path.join(root_path,df['audio_path'][i])
        transcript=df['bhashini_transcript'][i]
        audio=load_audio(audio_path)
        token_ids=model.tokenize(transcript)
        #preprocessed_transcript=transcript.replace(" ","|")

        emission=model.inference(audio)
        graph=compose_graph(emission,token_ids)

        t=graph.size(0)-1
        j=graph.size(1)-1

        t_list.append(t)
        j_list.append(j)
    
    return t_list,j_list


        

In [ ]:
filtered_df['token_length'],filtered_df['seq_length']=t_list,j_list

In [20]:
IPython.display.Audio(path,rate=8000)

In [29]:
transcript=filtered_df['bhashini_transcript'][54]

In [22]:
filtered_df['transcript'][63]

'आभार'

In [31]:
audio=load_audio(path)

In [32]:
audio_duration = len(audio) / SAMPLE_RATE


In [33]:
transcript_duration = len(transcript.split()) / 3


In [34]:
audio_duration

0.482625

In [35]:
transcript_duration

0.6666666666666666

In [1]:
# sample inspection 

In [43]:
import os 
sample_path=os.path.join(root_path,filtered_df['audio_path'][32])
model=_load_model()

In [44]:
audio=load_audio(sample_path)
transcript=filtered_df['bhashini_transcript'][32]
token_ids=model.tokenize(transcript)
preprocessed_transcript=transcript.replace(" ","|")


In [45]:
emission=model.inference(audio)
graph=compose_graph(emission,token_ids)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [46]:
#back tracking
t,j=graph.size(0)-1,graph.size(1)-1


In [47]:
t

10

In [48]:
j

11

In [49]:
blank_id=0

In [50]:
path=[Point(j,t,emission[t,0].exp().item())]


In [51]:
while j>0:
    
    assert t>0
    p_stay=emission[t-1,blank_id]
    p_change=emission[t-1,token_ids[j]]

    stayed=graph[t-1,j]+p_stay
    changed=graph[t-1,j-1]+p_change

    #stayed=graph[t-1,j]+p_stay
    #changed=graph[t-1,j-1]+p_change

    t-=1
    
    if changed>stayed:
            j -=1
    prob=(p_change if changed>stayed else p_stay).exp().item()
    path.append(Point(j,t,prob))

while t>0:
        prob=emission[t-1,blank_id].exp().item()
        path.append(Point(j,t-1,prob))
        t-=1
path=path[::-1]
   

AssertionError: 

In [31]:
path

[Point(token_index=0, time_index=0, score=0.14099103212356567),
 Point(token_index=1, time_index=1, score=0.32972249388694763),
 Point(token_index=2, time_index=2, score=2.2380454540252686),
 Point(token_index=3, time_index=3, score=1.3975948095321655),
 Point(token_index=4, time_index=4, score=16.56416130065918),
 Point(token_index=5, time_index=5, score=2.2606542110443115),
 Point(token_index=6, time_index=6, score=2.1871252059936523),
 Point(token_index=7, time_index=7, score=8.211030960083008),
 Point(token_index=8, time_index=8, score=39.40380859375)]

In [32]:
from data_preparation.alignment import merge_repeats

words=merge_repeats(path,preprocessed_transcript)

In [33]:
words

[श	(0.14): [    0,     1),
 ि	(0.33): [    1,     2),
 व	(2.24): [    2,     3),
 ि	(1.40): [    3,     4),
 र	(16.56): [    4,     5),
 |	(2.26): [    5,     6),
 म	(2.19): [    6,     7),
 े	(8.21): [    7,     8),
 ं	(39.40): [    8,     9)]

In [34]:
transcript

'शिविर में'

In [4]:
blah_df=pd.read_csv(os.path.join(root_path,'filtered_df.csv'))

In [5]:
df=blah_df[blah_df['seq_length']>blah_df['token_length']]

In [6]:
len(df)

1152

In [14]:
df=df.drop(columns=['level_0'])
df=df.reset_index()
df.head(20)


,level_0,Unnamed: 0,index,audio_path,transcript,duration,bhashini_transcript,negative_samples,token_length,seq_length
0,0,32,55,844424930944612-684-m_seg_7.wav,ग्रामीणों,3.210-3.692 sec,ग्रामीणों की,"('मुस्कुराते', '844424930897412-458-f_seg_2.wav')",10,11
1,1,56,94,844424930998970-1059-m_seg_4.wav,न्यूजीलैंड,2.044-2.565 sec,न्यूजीलैंड ने,"('स्थानांतरण', '844424930937106-394-f_seg_8.wav')",11,12
2,2,64,111,844424930523995-252-f_seg_5.wav,स्वागत,1.738-2.142 sec,स्वागत किया,"('हेमंत', '844424933459353-572-m_seg_0.wav')",8,10
3,3,76,131,844424932840389-727-f_seg_2.wav,पत्रकारों,1.171-1.574 sec,पत्रकारों से,"('नब्बे', '844424933477434-572-m_seg_5.wav')",8,11
4,4,103,170,844424932747251-645-f_seg_7.wav,संस्कार,2.216-2.659 sec,संस्कार किया,"('मैने', '844424931150104-186-m_seg_1.wav')",9,11
5,5,113,185,844424932758613-645-f_seg_7.wav,टेक्नोक्रेट्स,2.139-2.785 sec,एेक्नोक्रेट्स ने,"('गाँधी', '844424933264905-323-m_seg_4.wav')",14,15
6,6,115,187,844424933484264-266-m_seg_7.wav,रैंकिंग,5.379-5.780 sec,बैंकिंग में,"('विश्वकर्मा', '844424931206823-481-f_seg_5.wav')",8,10
7,7,148,238,844424931068385-819-f_seg_1.wav,पार्टियों,0.883-1.285 sec,पार्टियों प,"('दोस्त', '844424930931084-736-f_seg_5.wav')",8,10
8,8,178,284,844424932867379-727-f_seg_4.wav,कंपनियों,1.460-1.907 sec,कंपनियों को,"('कु', '844424931150123-186-m_seg_0.wav')",9,10
9,9,214,336,844424932608359-9-m_seg_3.wav,राष्ट्रपति,1.685-2.126 sec,रष्ट्रपति थे,"('बीस', '844424931018987-590-f_seg_0.wav')",9,11


In [28]:
import IPython.display
audio_path=os.path.join(root_path,df['audio_path'][21])
IPython.display.Audio(audio_path,rate=8000)

In [31]:
import math 
df_2=blah_df[blah_df['seq_length']-blah_df['token_length']==1]

In [33]:
df_2=df_2.drop(columns=['level_0'])
df_2=df_2.reset_index()

In [35]:
df_2.head(20)

,level_0,Unnamed: 0,index,audio_path,transcript,duration,bhashini_transcript,negative_samples,token_length,seq_length
0,32,32,55,844424930944612-684-m_seg_7.wav,ग्रामीणों,3.210-3.692 sec,ग्रामीणों की,"('मुस्कुराते', '844424930897412-458-f_seg_2.wav')",10,11
1,56,56,94,844424930998970-1059-m_seg_4.wav,न्यूजीलैंड,2.044-2.565 sec,न्यूजीलैंड ने,"('स्थानांतरण', '844424930937106-394-f_seg_8.wav')",11,12
2,113,113,185,844424932758613-645-f_seg_7.wav,टेक्नोक्रेट्स,2.139-2.785 sec,एेक्नोक्रेट्स ने,"('गाँधी', '844424933264905-323-m_seg_4.wav')",14,15
3,178,178,284,844424932867379-727-f_seg_4.wav,कंपनियों,1.460-1.907 sec,कंपनियों को,"('कु', '844424931150123-186-m_seg_0.wav')",9,10
4,331,331,504,844424931172033-590-f_seg_3.wav,पांचवें,1.126-1.528 sec,पांचवें से,"('प्रतिलिपि', '844424932335041-384-f_seg_9.wav')",8,9
5,348,348,528,844424932929205-961-f_seg_6.wav,प्रतिनिधियों,3.177-3.780 sec,प्रतिनिधियों से,"('राइजिंग', '844424931183245-481-f_seg_3.wav')",13,14
6,362,362,556,844424932868272-727-f_seg_6.wav,सुर्खियों,2.018-2.543 sec,सुर्खियों में,"('एयरवेज', '844424930862333-1177-f_seg_6.wav')",11,12
7,374,374,575,844424931190367-590-f_seg_1.wav,रोगियों,0.486-0.891 sec,रोगियों को,"('निर्मल', '844424932482745-1102-f_seg_0.wav')",8,9
8,393,393,603,844424932717726-126-m_seg_9.wav,प्रयोग,5.969-6.410 sec,प्रयोग किया,"('घोष', '844424932833031-742-f_seg_0.wav')",9,10
9,410,410,627,844424932608359-9-m_seg_1.wav,अमेरिका,1.123-1.524 sec,अमेरिका के,"('होटलों', '844424930653291-939-f_seg_5.wav')",8,9


In [46]:
audio_path_2=os.path.join(root_path,df_2['audio_path'][30])
IPython.display.Audio(audio_path_2,rate=8000)


In [44]:
df_2['transcript'][30]

'सस्पेंड'

In [45]:
df_2['bhashini_transcript'][30]

'सस्पेंड किया'

In [47]:
len(df_2)

722

In [53]:
len(correct_df)

7138

In [54]:
same_df=blah_df[blah_df['token_length']==blah_df['seq_length']]

In [55]:
len(same_df)

1578

In [56]:
len(blah_df)

21869

In [58]:
same_df=same_df.drop(columns=['level_0'])
same_df=same_df.reset_index()

In [72]:
same_df.head(30)

,level_0,Unnamed: 0,index,audio_path,transcript,duration,bhashini_transcript,negative_samples,token_length,seq_length
0,6,6,10,844424930647555-939-f_seg_1.wav,महीने,0.926-1.328 sec,महीने की<unk>,"('हलूरी', '844424931088629-1148-f_seg_1.wav')",8,8
1,21,21,42,844424933495501-705-m_seg_7.wav,दक्षिण,3.892-4.333 sec,दक्षिण में,"('शॉर्ट्स', '844424930741372-684-m_seg_4.wav')",9,9
2,29,29,52,844424932710748-645-f_seg_3.wav,समृद्धि,1.134-1.620 sec,समृद्धि की<unk>,"('हवा', '844424932360870-384-f_seg_0.wav')",10,10
3,33,33,56,844424932904916-1128-f_seg_6.wav,प्रदूषण,2.023-2.468 sec,प्रदूषण के,"('ज्योतिषीय', '844424931120994-819-f_seg_0.wav')",9,9
4,42,42,68,844424932811593-346-f_seg_6.wav,मरीजों,2.764-3.165 sec,मरीजों की,"('आवेदक', '844424930865055-229-f_seg_0.wav')",8,8
5,43,43,71,844424930851560-252-f_seg_1.wav,शिविर,0.241-0.642 sec,शिविर में,"('लारा', '844424930708832-727-f_seg_0.wav')",8,8
6,57,57,96,844424933498939-1023-m_seg_6.wav,दौरान,3.327-3.768 sec,दौरान किया,"('संभाली', '844424930791828-229-f_seg_9.wav')",9,9
7,70,70,123,844424932854240-682-f_seg_3.wav,पार्टियां,2.542-3.066 sec,पार्टियां इस,"('लोग', '844424932581458-645-f_seg_0.wav')",11,11
8,82,82,139,844424932906812-727-f_seg_3.wav,भागलपुर,1.484-1.965 sec,भागलपुर में,"('दीमापुर', '844424930613534-252-f_seg_10.wav')",10,10
9,96,96,160,844424930583706-946-f_seg_4.wav,मीडिया,2.250-2.652 sec,मीडिया थी,"('घाट', '844424933476302-390-m_seg_1.wav')",8,8


In [76]:
audio_path_3=os.path.join(root_path,same_df['audio_path'][29])
IPython.display.Audio(audio_path_3,rate=8000)


In [77]:
less_df=blah_df[blah_df['token_length']>blah_df['seq_length']]
less_df=less_df.drop(columns=['level_0'])
less_df=less_df.reset_index()

In [78]:
less_df.head(30)

,level_0,Unnamed: 0,index,audio_path,transcript,duration,bhashini_transcript,negative_samples,token_length,seq_length
0,0,0,0,844424932709934-468-f_seg_10.wav,प्रतिष्ठा,6.099-6.982 sec,प्रतिष्ठा को,"('उलटफेर', '844424932761946-3-f_seg_10.wav')",20,11
1,1,1,1,844424930725606-329-f_seg_0.wav,शुक्रवार,0.000-0.928 sec,शुक्रवार को,"('हेक्टेयर', '844424931023729-142-m_seg_9.wav')",21,10
2,2,2,2,844424931275037-1102-f_seg_0.wav,तेंदुलकर,0.000-0.882 sec,तेंदुलकर ने,"('सुरक्षा', '844424932717790-126-m_seg_3.wav')",20,10
3,3,3,3,844424931088775-289-f_seg_6.wav,मामले,2.407-2.808 sec,ममले पर,"('प्रशासन', '844424930614601-229-f_seg_4.wav')",8,6
4,4,4,6,844424932400848-384-f_seg_0.wav,राशिद,0.000-1.327 sec,राशि ती,"('प्राथमिक', '844424930926413-229-f_seg_0.wav')",31,6
5,5,5,8,844424930578539-1128-f_seg_0.wav,वह,0.000-0.688 sec,वह ए,"('ब्याज', '844424930583714-946-f_seg_0.wav')",15,3
6,7,7,11,844424932927416-1102-f_seg_4.wav,शोभा,1.980-2.385 sec,शोभा से,"('प्रभावशाली', '844424933506917-905-m_seg_6.wav')",8,6
7,8,8,15,844424930618241-326-f_seg_7.wav,कुमार,4.414-4.896 sec,कुमार से,"('साधा', '844424931159107-590-f_seg_10.wav')",10,7
8,9,9,16,844424931109777-458-f_seg_4.wav,ठिकाने,1.931-2.373 sec,ठिकाने लग,"('कोवलम', '844424933521804-1023-m_seg_0.wav')",9,8
9,10,10,17,844424930667540-229-f_seg_0.wav,उस,0.000-0.803 sec,उस व,"('अकाली', '844424930545059-229-f_seg_10.wav')",18,3


In [79]:
len(less_df)

19139

In [86]:
audio_path_4=os.path.join(root_path,less_df['audio_path'][18])
IPython.display.Audio(audio_path_4,rate=8000)